In [1]:
from cftime import DatetimeNoLeap
from metpy import calc, units
from os import chdir, system
from numpy import empty, float64, full
from xarray import DataArray, open_dataset

In [2]:
nc_path = '/home/zhangc/scenariomip_cmip6/nc_data/'
im_path = '/home/zhangc/scenariomip_cmip6/im_data/'
os_path = '/home/zhangc/repositories/nc2im_cesm2/'

In [3]:
chdir(nc_path)

In [4]:
tnum = 2015, 1, 1, 0
t6hr = DatetimeNoLeap(tnum[0], tnum[1], tnum[2], tnum[3])
tmon = DatetimeNoLeap(tnum[0], tnum[1], 15, 12)
file_time = str(t6hr).replace(' ', '_')

In [5]:
nc_con = ['fracdata_0.9x1.25_gx1v6_c090317.nc',
          'USGS-gtopo30_0.9x1.25_remap_c051027.nc']

In [6]:
nc_6hr = ['ta_6hrLev_CESM2_ssp245_r2i1p1f1_gn_201501010000-202412311800.nc',
          'hus_6hrLev_CESM2_ssp245_r2i1p1f1_gn_201501010000-202412311800.nc',
          'ua_6hrLev_CESM2_ssp245_r2i1p1f1_gn_201501010000-202412311800.nc',
          'va_6hrLev_CESM2_ssp245_r2i1p1f1_gn_201501010000-202412311800.nc',
          'ps_6hrLev_CESM2_ssp245_r2i1p1f1_gn_201501010000-202412311800.nc']

In [7]:
nc_mon = ['ts_Amon_CESM2_ssp245_r2i1p1f1_gn_201501-206412.nc']

In [8]:
das = []

In [9]:
for nc in nc_6hr:
    ds = open_dataset(nc).sel(time=t6hr)
    vi = ds.variable_id
    da = ds[vi]
    das.append(da)
    file_name = vi + '_' + file_time + '.nc'
    da.to_netcdf(im_path+file_name)

/home/zhangc/miniconda3/envs/python3/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'ta' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/home/zhangc/miniconda3/envs/python3/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'hus' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/home/zhangc/miniconda3/envs/python3/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'ua' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/home/zhangc/miniconda3/envs/python3/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'va' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/home/zhangc/miniconda3/envs/python3/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarnin

In [10]:
t2 = das[0].isel(lev=-1)
t2.name = 't2'
file_name = t2.name + '_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

In [11]:
q2 = das[1].isel(lev=-1)
q2.name = 'q2'
file_name = q2.name + '_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

In [12]:
u2 = das[2].isel(lev=-1)
u2.name = 'u2'
file_name = u2.name + '_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

In [13]:
v2 = das[3].isel(lev=-1)
v2.name = 'v2'
file_name = v2.name + '_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

In [14]:
da = das[0]
lat = da.lat.values
lon = da.lon.values
lev = da.lev.values

In [15]:
zeros = empty(da.shape)

In [16]:
for i in range(len(lev)):
    zeros[i] = full(da.shape[1:3], lev[i])

In [17]:
p = DataArray(name='p', data=zeros, coords=[lev, lat, lon], dims=['lev', 'lat', 'lon'])

In [18]:
file_name = 'p_' + file_time + '.nc'
p.to_netcdf(im_path+file_name)

In [19]:
p2 = p.isel(lev=-1)
p2.name = 'p2'
file_name = p2.name + '_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

In [20]:
for nc in nc_mon:
    ds = open_dataset(nc).sel(time=tmon)
    vi = ds.variable_id
    da = ds[vi]
    file_name = vi + '_' + file_time + '.nc'
    da.to_netcdf(im_path+file_name)

/home/zhangc/miniconda3/envs/python3/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'ts' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,


In [21]:
chdir(os_path)

In [22]:
command = 'ncl convert_nc_to_im.ncl ' + "'file_time=" + '"' + file_time + '"' + "'"

In [23]:
#system(command)

In [ ]:
from numpy import array

In [ ]:
K = units.units.kelvin
Pa = units.units.Pa

In [ ]:
Q2 = q2.values
T2 = t2.values*K
P2 = p2.values*Pa

In [ ]:
R2 = array(calc.relative_humidity_from_specific_humidity(Q2, T2, P2))
r2 = DataArray(name='r2', data=R2, coords=[lat, lon], dims=['lat', 'lon'])
file_name = p2.name + '_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)